In [91]:
import pandas as pd

In [92]:
df = pd.read_csv('final_results_by_BERT.csv')

In [93]:
df.head()

,review_text,aspect_by_BERT,sentiment,sentiment_strength,product_name,product_type,rating,initial_sentiment,normal_sentiment_score,normal_sentiment,rating_by_normal_sentiment,aspect_ratings,aspect_rating_by_BERT_sentiment
0,dr oz accomplished heart surgeon field cardiac...,['dean ornish'],['Positive'],"[[0.0006767544546164572, 0.0004284284368623048...",healing from the heart,books,4,positive,0.9300,positive,5,"{'transplantation': 4, 'epilogue': 4, 'medicin...",{'dean ornish': 4}
1,gorgeous artwork comic books contains extraord...,['artwork'],['Positive'],"[[0.003452055621892214, 0.00021875665697734803...",mythology,books,5,positive,0.9118,positive,5,"{'books': 4, 'artwork': 3}",{'artwork': 4}
2,going short sweet review many others commented...,['picture'],['Positive'],"[[0.07295969128608704, 0.0005088011967018247, ...",guns germs and steel,books,5,positive,0.9912,positive,5,"{'review': 4, 'others': 2, 'value': 3, 'though...",{'picture': 4}
3,since worlds fair fanatic read almost everythi...,['interior'],['Positive'],"[[0.37773197889328003, 0.003444268833845854, 0...",worlds fairs and the end of progress,books,4,positive,0.9633,positive,5,"{'worlds': 4, 'read': 4, 'book_author': 3, 'er...",{'interior': 4}
4,one book taught use slr cameras manual itself ...,"['lenses', 'fit']","['Negative', 'Positive']","[[0.9923324584960938, 0.003380761481821537, 0....",teach yourself photography,books,4,positive,0.9584,positive,5,"{'arrangement_chapters': 3, 'way': 4, 'lenses'...","{'lenses': 1, 'fit': 4}"


In [94]:
from sklearn.metrics import accuracy_score

In [95]:
ground_truth = df['rating']
rating_by_nlp_result = df['rating_by_normal_sentiment']

In [96]:
acc = accuracy_score(ground_truth,rating_by_nlp_result)

### Baseline Accuracy

In [97]:
baseline_results = pd.read_excel('baseline_results.xlsx')
baseline_results_ground_truth = baseline_results['initial_sentiment']
baseline_results_result = baseline_results['normal_sentiment']

acc_baseline = accuracy_score(baseline_results_ground_truth,baseline_results_result)
print('Baseline Accuracy',acc_baseline)

Baseline Accuracy 0.7307


In [98]:
baseline_results.columns,df.columns

(Index(['Unnamed: 0', 'product_name', 'review_text', 'rating',
        'initial_sentiment', 'normal_sentiment_score', 'normal_sentiment',
        'rating_by_normal_sentiment', 'aspect_ratings'],
       dtype='object'),
 Index(['review_text', 'aspect_by_BERT', 'sentiment', 'sentiment_strength',
        'product_name', 'product_type', 'rating', 'initial_sentiment',
        'normal_sentiment_score', 'normal_sentiment',
        'rating_by_normal_sentiment', 'aspect_ratings',
        'aspect_rating_by_BERT_sentiment'],
       dtype='object'))

### Accuracy on Improved NLP based sentiment extraction

In [99]:
df = pd.read_csv('final_results_by_BERT.csv')
final_results_ground_truth = df['initial_sentiment']
final_results_pred = df['normal_sentiment']

acc_final = accuracy_score(final_results_ground_truth,final_results_pred)
print('Final Model NLP(technique based)',acc_final)

Final Model NLP(technique based) 0.9538780343398461


In [100]:
row = df.iloc[4]

In [101]:
row

review_text                        one book taught use slr cameras manual itself ...
aspect_by_BERT                                                     ['lenses', 'fit']
sentiment                                                   ['Negative', 'Positive']
sentiment_strength                 [[0.9923324584960938, 0.003380761481821537, 0....
product_name                                              teach yourself photography
product_type                                                                   books
rating                                                                             4
initial_sentiment                                                           positive
normal_sentiment_score                                                        0.9584
normal_sentiment                                                            positive
rating_by_normal_sentiment                                                         5
aspect_ratings                     {'arrangement_chapters': 3, 'w

In [102]:
def calculate_bert_sentiment(row):
    positive_score = 0
    negative_score = 0
    neutral_score  = 0

    positive_count = 0
    negative_count = 0
    neutral_count = 0

    for index,sentiment in enumerate(eval(row['sentiment'])):
        sentiment_scores = eval(row['sentiment_strength'])[index]

        if sentiment == 'Positive':
            positive_score = positive_score + sentiment_scores[2]
            positive_count+=1
        if sentiment == 'Negative':
            negative_score = negative_score + sentiment_scores[0]
            negative_count+=1
        else:
            neutral_score = neutral_score + sentiment_scores[1]
            neutral_count+=1
        
    if positive_count > 0:
        positive_score = positive_score/positive_count
    
    if negative_count > 0:
        negative_score = negative_score/negative_count
    
    if neutral_count > 0:
        neutral_score = negative_score/neutral_count

    if positive_score == max(positive_score,negative_score,neutral_score):
        return 'Positive'
    if negative_score == max(positive_score,negative_score,neutral_score):
        return 'Negative'
    if neutral_score == max(positive_score,negative_score,neutral_score):
        return 'Neutral'

In [103]:
bert_sentiments = df.apply(calculate_bert_sentiment,axis='columns')

### Accuracy on Sentiment Extraction Bert based

In [104]:
df = pd.read_csv('final_results_by_BERT.csv')
final_results_ground_truth = df['initial_sentiment']

acc_final_bert = accuracy_score(final_results_ground_truth,bert_sentiments)
print('Final model Bert Based',acc_final)

Final model Bert Based 0.9538780343398461
